# SpiNNaker Synfire Example
This example runs a simple "Synfire Chain" on the SpiNNaker platform, that is a chain of Populations of neurons.  When a neuron in a Population in the chain spikes, it sends the spike to all the neurons in the next Population, which then spikes, and so on, with the last Population in the chain connected to first.  An initial spike is injected into the first Population at the start of the simulation which targets all neurons in the Population, and then this behaviour continues until the simulation is stopped.  Finally, a graph is drawn showing the resulting spikes of each neuron, in each Population, with the spikes from each Population coloured differently.  A delay of 17 timesteps between each spike is also specified, so the lines of spikes over time are sloped.

## Running the Example
To run the example, please execute the cells below (Run > Run All cells), or press the ![Restart Kernel and Run All Cells](data:image/svg+xml,%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20width%3D%2224%22%20height%3D%2224%22%20viewBox%3D%220%200%2024%2024%22%20data-icon%3D%22ui-components%3Afast-forward%22%3E%3Cg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20class%3D%22jp-icon3%22%20fill%3D%22%23616161%22%3E%3Cpath%20d%3D%22M4%2018l8.5-6L4%206v12zm9-12v12l8.5-6L13%206z%22%3E%3C%2Fpath%3E%3C%2Fg%3E%3C%2Fsvg%3E) button above, and select "Restart".

## Note
If you have cloned this repository automatically following a demonstration link, any changes made will not be persisted.  If you would like to make more persistent changes, please create an EBRAINS Collaboratory, where you can manually clone this repository and make changes.  See more details [here](https://wiki.ebrains.eu/bin/view/Collabs/neuromorphic/Getting%20access/)

In [ ]:
# We have to use it this way until the next PyNN release
import pyNN.spiNNaker as pynn

# Set up some constants
n_neurons = 100
n_populations = 10
weights = 0.5
delays = 17.0
simtime = 1000

# Set up pyNN with a timestep of 1.0ms
pynn.setup(timestep=1.0)

# Set up a Population to start the synfire chain
stimulus = pynn.Population(1, pynn.SpikeSourceArray(spike_times=[0]), label='stimulus')

# Set up the Synfire chain populations
chain_pops = [
    pynn.Population(n_neurons, pynn.IF_curr_exp(), label='chain_{}'.format(i))
    for i in range(n_populations)
]

# Record the spikes in the Populations so we can see them at the end!
for pop in chain_pops:
    pop.record("spikes")
    
# Connect them together
connector = pynn.FixedNumberPreConnector(10)
for i in range(n_populations):
    pynn.Projection(
        chain_pops[i], chain_pops[(i + 1) % n_populations], 
        connector, synapse_type=pynn.StaticSynapse(weight=weights, delay=delays))
    
# Connect the simulus to start the chain
pynn.Projection(stimulus, chain_pops[0], pynn.AllToAllConnector(), synapse_type=pynn.StaticSynapse(weight=5.0))

# Run the Simulation
pynn.run(simtime)

# Extract the data
neo_data = [pop.get_data("spikes") for pop in chain_pops]

spike_trains = [neo.segments[0].spiketrains for neo in neo_data]
offset = 0
for spike_train_list in spike_trains:
    for spike_train in spike_train_list:
        spike_train.annotations["source_index"] += offset
    offset += n_neurons

# End the simulation
pynn.end()

In [ ]:
# Plot the results
import matplotlib.pyplot as plt
import numpy
import pyNN.utility.plotting as plotting
%matplotlib inline

colours = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

plt.figure()
plt.xlim(0, simtime)
plt.ylim(0, n_neurons * n_populations)
plt.xlabel("Time (ms)")
plt.ylabel("Neuron ID")
plt.title("Synfire Neurons")
offset = 0
for i, pop_spikes in enumerate(spike_trains):
    colour = colours[i % len(colours)]
    for spike_train in pop_spikes:
        neuron_id = spike_train.annotations['source_index'] + offset
        plt.plot(spike_train, numpy.ones_like(spike_train) * neuron_id, ".", color=colour)
    offset += len(spike_train)
plt.show()